First step is to read, and preprocess the data.

In [ ]:
%tensorflow_version 1.x
import tensorflow

In [ ]:
#imports
import os
import time
from google.colab import drive
import numpy as np
import matplotlib as plt
from matplotlib import pyplot
import pandas as pd

# Import keras modules
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
import keras
from keras import models, layers, Input, regularizers
from keras.utils import np_utils
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from keras.layers import Dense
import random
random.seed(42)

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/MRP/Covid-target/self/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Toggles
self_sup = True
incep = False
aug = True
saves = False
big_arch = 0
file_name = "Exp4v3p1_full"

In [ ]:
start_time = time.clock()


In [ ]:
#I will try Inception, so change H,W
if incep == True:
    H=299
    W=299
else:
    H=224
    W=224



In [ ]:
#End of preprocessing

In [ ]:
if incep == True:
    base_model = keras.applications.InceptionV3(
        include_top = False,
        weights = "imagenet")
else:
    base_model = keras.applications.DenseNet169(
        include_top = False,
        weights = "imagenet")    

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


51879936/51877672 [==============================] - 5s 0us/step


Next step is to load the pre-trained model, add our own dense layer on top, and train the dense layer.

In [ ]:
if self_sup == True:
    cp = ModelCheckpoint(filepath= file_name + '_run1.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [ ]:
if aug == True:
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
                                                dtype='float64')

    #on test, we do not augment, merely preprocess
    test_datagen = keras.preprocessing.image.ImageDataGenerator(
                                                dtype='float64')
    
    train_generator = train_datagen.flow_from_directory(
        'train',
        target_size = (W,H),
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        'test',
        target_size = (W,H),
        class_mode = 'categorical'
    )

Found 1086 images belonging to 2 classes.
Found 406 images belonging to 2 classes.


In [ ]:
#no fine tuning

x = base_model.output
x = GlobalAveragePooling2D()(x)

if big_arch == 1:
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.01),)(x) #0.05
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.01),)(x) #0.05
    x = layers.Dropout(0.5)(x)    

else:
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.01),)(x) #0.05
    x = layers.Dropout(0.5)(x)

predictions = Dense(2, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


if self_sup == True:
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=["accuracy"])

    if aug == True:
        hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp], verbose=1)
    else:
        hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp], verbose=1)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/30
12/34 [=========>....................] - ETA: 7:15 - loss: 12.9281 - accuracy: 0.4922

/tensorflow-1.15.2/python3.6/keras/utils/data_utils.py:616: UserWarning: The input 25 could not be retrieved. It could be because a worker has died.
  UserWarning)


34/34 [==============================] - 719s 21s/step - loss: 12.3259 - accuracy: 0.4779 - val_loss: 12.7371 - val_accuracy: 0.5012

Epoch 00001: val_loss improved from inf to 12.73712, saving model to Exp4v3p1_full_run1.hdf5
Epoch 2/30
34/34 [==============================] - 5s 154ms/step - loss: 10.7097 - accuracy: 0.5276 - val_loss: 11.0532 - val_accuracy: 0.5099

Epoch 00002: val_loss improved from 12.73712 to 11.05317, saving model to Exp4v3p1_full_run1.hdf5
Epoch 3/30
34/34 [==============================] - 6s 169ms/step - loss: 9.3799 - accuracy: 0.5272 - val_loss: 10.3665 - val_accuracy: 0.5049

Epoch 00003: val_loss improved from 11.05317 to 10.36650, saving model to Exp4v3p1_full_run1.hdf5
Epoch 4/30
34/34 [==============================] - 6s 171ms/step - loss: 8.2451 - accuracy: 0.5350 - val_loss: 8.8843 - val_accuracy: 0.5074

Epoch 00004: val_loss improved from 10.36650 to 8.88426, saving model to Exp4v3p1_full_run1.hdf5
Epoch 5/30
34/34 [==============================

In [ ]:
if self_sup == True:
    cp2 = ModelCheckpoint(filepath= file_name + '_run2.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [ ]:
if aug == True:
    train_generator = train_datagen.flow_from_directory(
        'train',
        target_size = (W,H),
        batch_size = 4,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        'test',
        target_size = (W,H),
        class_mode = 'categorical'
    )

Found 1086 images belonging to 2 classes.
Found 406 images belonging to 2 classes.


In [ ]:
if self_sup == True:
    #Fine tuning

    from keras.models import load_model
    model = load_model(file_name + '_run1.hdf5')

    for layer in model.layers:
      layer.trainable = True

    from keras.optimizers import SGD
    model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='binary_crossentropy', metrics=["accuracy"])

    if aug == True:
        hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp2], verbose=1)
    else:
        hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp2], verbose=1)

    

Epoch 1/30
272/272 [==============================] - 83s 305ms/step - loss: 1.9024 - accuracy: 0.6436 - val_loss: 1.9250 - val_accuracy: 0.5456

Epoch 00001: val_loss improved from inf to 1.92495, saving model to Exp4v3p1_full_run2.hdf5
Epoch 2/30
272/272 [==============================] - 49s 180ms/step - loss: 1.8599 - accuracy: 0.6699 - val_loss: 1.9744 - val_accuracy: 0.5640

Epoch 00002: val_loss did not improve from 1.92495
Epoch 3/30
272/272 [==============================] - 49s 181ms/step - loss: 1.8429 - accuracy: 0.6952 - val_loss: 1.9589 - val_accuracy: 0.5924

Epoch 00003: val_loss did not improve from 1.92495
Epoch 4/30
272/272 [==============================] - 50s 184ms/step - loss: 1.8333 - accuracy: 0.7192 - val_loss: 1.9175 - val_accuracy: 0.5998

Epoch 00004: val_loss improved from 1.92495 to 1.91749, saving model to Exp4v3p1_full_run2.hdf5
Epoch 5/30
272/272 [==============================] - 49s 181ms/step - loss: 1.8103 - accuracy: 0.7159 - val_loss: 1.9723 - va

In [ ]:
if self_sup == True:
    #learning rate decay
    
    lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    cp3 = ModelCheckpoint(filepath=file_name + '_run3.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [ ]:
if self_sup == True:
    model = load_model(file_name + '_run2.hdf5')
    model.compile(optimizer=SGD(lr=0.000001), loss='binary_crossentropy', metrics=["accuracy"])

    if aug == True:
        hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp3], verbose=1)
    else:
        hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp3], verbose=1)

Epoch 1/30
272/272 [==============================] - 84s 309ms/step - loss: 1.5452 - accuracy: 0.8844 - val_loss: 1.7625 - val_accuracy: 0.7796

Epoch 00001: val_loss improved from inf to 1.76252, saving model to Exp4v3p1_full_run3.hdf5
Epoch 2/30
272/272 [==============================] - 50s 182ms/step - loss: 1.5389 - accuracy: 0.8895 - val_loss: 1.8341 - val_accuracy: 0.7771

Epoch 00002: val_loss did not improve from 1.76252
Epoch 3/30
272/272 [==============================] - 50s 186ms/step - loss: 1.5362 - accuracy: 0.8918 - val_loss: 1.7379 - val_accuracy: 0.7734

Epoch 00003: val_loss improved from 1.76252 to 1.73790, saving model to Exp4v3p1_full_run3.hdf5
Epoch 4/30
272/272 [==============================] - 50s 183ms/step - loss: 1.5471 - accuracy: 0.8734 - val_loss: 1.7597 - val_accuracy: 0.7759

Epoch 00004: val_loss did not improve from 1.73790
Epoch 5/30
272/272 [==============================] - 50s 183ms/step - loss: 1.5373 - accuracy: 0.8863 - val_loss: 1.5678 - va

In [ ]:
#Try on benign vs malignant
os.chdir('/content/drive/My Drive/MRP/Covid-target/organized/')
print()

In [ ]:
org_base = '/content/drive/My Drive/MRP/Covid-target/self/'

In [ ]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
cp4 = ModelCheckpoint(filepath=file_name + '_run4.hdf5', verbose=1, monitor='val_accuracy', save_best_only=True)

In [ ]:
if aug == True:
    train_generator = train_datagen.flow_from_directory(
        'big_train',
        target_size = (W,H),
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        'test',
        target_size = (W,H),
        class_mode = 'categorical'
    )


Found 543 images belonging to 2 classes.
Found 203 images belonging to 2 classes.


In [ ]:
#benign vs malignant
if self_sup == True:
    model = load_model(org_base + file_name + '_run3.hdf5')


for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=["accuracy"])

if aug == True:
    hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp4], verbose=1)
else:
    hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), callbacks=[cp4], verbose=1)

Epoch 1/30
17/17 [==============================] - 267s 16s/step - loss: 1.6460 - accuracy: 0.7974 - val_loss: 2.4798 - val_accuracy: 0.6182

Epoch 00001: val_accuracy improved from -inf to 0.61823, saving model to Exp4v3p1_full_run4.hdf5
Epoch 2/30
17/17 [==============================] - 6s 375ms/step - loss: 1.2320 - accuracy: 0.9880 - val_loss: 1.7130 - val_accuracy: 0.7094

Epoch 00002: val_accuracy improved from 0.61823 to 0.70936, saving model to Exp4v3p1_full_run4.hdf5
Epoch 3/30
17/17 [==============================] - 6s 375ms/step - loss: 1.1292 - accuracy: 0.9982 - val_loss: 1.7722 - val_accuracy: 0.7241

Epoch 00003: val_accuracy improved from 0.70936 to 0.72414, saving model to Exp4v3p1_full_run4.hdf5
Epoch 4/30
17/17 [==============================] - 6s 371ms/step - loss: 1.0069 - accuracy: 1.0000 - val_loss: 1.1042 - val_accuracy: 0.7709

Epoch 00004: val_accuracy improved from 0.72414 to 0.77094, saving model to Exp4v3p1_full_run4.hdf5
Epoch 5/30
17/17 [=============

In [ ]:
if aug == True:
    train_generator = train_datagen.flow_from_directory(
        'train',
        target_size = (W,H),
        batch_size = 4,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        'test',
        target_size = (W,H),
        class_mode = 'categorical'
    )

Found 425 images belonging to 2 classes.
Found 203 images belonging to 2 classes.


In [ ]:
cp5 = ModelCheckpoint(filepath=file_name + '_run5.hdf5', verbose=1, monitor='val_accuracy', save_best_only=True)

In [ ]:
#model = load_model(file_name + '_run4.hdf5')
for layer in model.layers:
   layer.trainable = True

from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss='binary_crossentropy', metrics=["accuracy"])

if aug == True:
    hist = model.fit_generator(train_generator, epochs=30, validation_data=test_generator, callbacks=[cp5], verbose=1)
else:
    hist1 = model.fit(x=train_img,y = y_train, epochs=30, validation_data=(test_img, y_test), verbose=1)

Epoch 1/30
107/107 [==============================] - 275s 3s/step - loss: 0.2826 - accuracy: 0.9341 - val_loss: 0.9508 - val_accuracy: 0.8079

Epoch 00001: val_accuracy improved from -inf to 0.80788, saving model to Exp4v3p1_full_run5.hdf5
Epoch 2/30
107/107 [==============================] - 21s 193ms/step - loss: 0.3415 - accuracy: 0.9294 - val_loss: 1.6054 - val_accuracy: 0.8325

Epoch 00002: val_accuracy improved from 0.80788 to 0.83251, saving model to Exp4v3p1_full_run5.hdf5
Epoch 3/30
107/107 [==============================] - 21s 196ms/step - loss: 0.3218 - accuracy: 0.9212 - val_loss: 1.4836 - val_accuracy: 0.8424

Epoch 00003: val_accuracy improved from 0.83251 to 0.84236, saving model to Exp4v3p1_full_run5.hdf5
Epoch 4/30
107/107 [==============================] - 21s 195ms/step - loss: 0.2602 - accuracy: 0.9271 - val_loss: 1.0016 - val_accuracy: 0.8473

Epoch 00004: val_accuracy improved from 0.84236 to 0.84729, saving model to Exp4v3p1_full_run5.hdf5
Epoch 5/30
107/107 [=

In [ ]:
total_time = time.clock() - start_time
print(total_time)

9180.571982


In [ ]:
len(os.listdir('test/non-covid/'))

105

In [ ]:
model = load_model(file_name + '_run5.hdf5')

In [ ]:
generator = test_datagen.flow_from_directory(
        'test',
        target_size=(W, H),
        batch_size=1,
        class_mode=None,  # only data, no labels
        shuffle=False)  # keep data in same order as labels

probabilities = model.predict_generator(generator, 203)

Found 203 images belonging to 2 classes.


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, accuracy_score

y_true = np.array([0] * 98 + [1] * 105)
y_pred = np.argmax(probabilities, axis = 1)

confusion_matrix(y_true, y_pred.round())

array([[83, 15],
       [11, 94]])

In [ ]:
print(f1_score(y_true, y_pred))
print()
print(roc_auc_score(y_true, y_pred))
print()
print(accuracy_score(y_true, y_pred))

0.8785046728971964

0.8710884353741496

0.8719211822660099
